In [1]:
import pandas as pd

In [2]:
boardmember_count = pd.read_csv('organization_summary.csv')
connectivity = pd.read_csv('raw_metrics_feb26.csv')
boardmember_count

,rowtype,companyid,annualreportdate,boardname,boardid,ticker,directorname,networksize,numberdirectors
0,Board Member,10925,2012-12-01,EQUITY ONE INC (De-listed 03/2017),10925,EQY,David Fischel,1107.0,10.0
1,Board Member,10925,2013-12-01,EQUITY ONE INC (De-listed 03/2017),10925,EQY,David Fischel,1107.0,9.0
2,Board Member,10925,2011-12-01,EQUITY ONE INC (De-listed 03/2017),10925,EQY,David Fischel,1107.0,10.0
3,Board Member,10925,2015-12-01,EQUITY ONE INC (De-listed 03/2017),10925,EQY,David Fischel,1107.0,9.0
4,Board Member,10925,2014-12-01,EQUITY ONE INC (De-listed 03/2017),10925,EQY,David Fischel,1107.0,9.0
...,...,...,...,...,...,...,...,...,...
1337263,Board Member,2652673,2024-12-01,Upgrade Inc (Universal Credit),2652673,NaN,Doctor Joseph Breeden,31.0,6.0
1337264,Board Member,2347715,2024-12-01,Graylog Inc,2347715,NaN,Maury Domengeaux,14.0,6.0
1337265,Board Member,3119645,2024-12-01,Kubit AI Inc,3119645,NaN,Lake Dai,9.0,4.0
1337266,Board Member,2511894,2024-12-01,FrenchFounders Inc,2511894,NaN,Benoit Buridant,15.0,6.0


In [3]:
count_mismatched = (boardmember_count['boardid'] != boardmember_count['companyid']).sum()
print(count_mismatched)

0


In [4]:
boardmember_count = boardmember_count.groupby(['companyid', 'annualreportdate'])['numberdirectors'].mean().reset_index()

In [5]:
boardmember_count['annualreportdate'] = pd.to_datetime(boardmember_count['annualreportdate'], errors='coerce')
boardmember_count['annualreportdate'] = boardmember_count['annualreportdate'].dt.year.astype('Int64')  # Nullable integer
connectivity['year'] = pd.to_numeric(connectivity['year'], errors='coerce').astype('Int64')

In [6]:
boardmember_count = boardmember_count.rename(columns={'annualreportdate':'year'})

In [7]:
# Number of directors
print(boardmember_count['numberdirectors'].min())
print(boardmember_count['numberdirectors'].max())

1.0
105.0


In [8]:
nulls = boardmember_count.groupby('companyid')['numberdirectors'].apply(lambda x: x.isna().sum())
nulls = nulls.sort_values(ascending=False)
nulls.head(5)

companyid
875444     13
1974896     9
1790774     9
1075248     7
818269      7
Name: numberdirectors, dtype: int64

In [9]:
connectivity.dtypes

company_linking      int64
year                 Int64
betweenness        float64
closeness          float64
degree             float64
eigenvector        float64
dtype: object

In [10]:
boardmember_count

,companyid,year,numberdirectors
0,3,2003,9.0
1,3,2004,8.0
2,3,2005,7.0
3,3,2006,8.0
4,6,2003,9.0
...,...,...,...
150808,3915440,2024,3.0
150809,3918012,2025,4.0
150810,3919247,2025,6.0
150811,3920262,2025,4.0


In [11]:
connectivity

,company_linking,year,betweenness,closeness,degree,eigenvector
0,11608,1968,0.000000,0.000000,0.000000,1.000000e+00
1,16506,1968,0.000000,0.000000,0.000000,1.000000e+00
2,21015,1968,0.000000,1.000000,0.000000,5.238103e-17
3,21515,1968,0.000000,1.000000,0.000000,0.000000e+00
4,2366,1968,0.000000,0.000000,0.000000,9.999853e-01
...,...,...,...,...,...,...
118758,9831,2025,0.000000,0.056709,0.022222,5.905482e-03
118759,9834,2025,0.000035,0.073471,0.022222,1.935864e-02
118760,9931,2025,0.001887,0.045431,0.044444,1.955908e-03
118761,9940,2025,0.080838,0.095844,0.244444,1.407078e-01


In [12]:
print(connectivity.dtypes)
print(boardmember_count.dtypes)


company_linking      int64
year                 Int64
betweenness        float64
closeness          float64
degree             float64
eigenvector        float64
dtype: object
companyid            int64
year                 Int64
numberdirectors    float64
dtype: object


In [13]:
df_merged = pd.merge(connectivity, boardmember_count, left_on=['company_linking', 'year'], right_on=['companyid', 'year'], how='left')

In [14]:
df_merged

,company_linking,year,betweenness,closeness,degree,eigenvector,companyid,numberdirectors
0,11608,1968,0.000000,0.000000,0.000000,1.000000e+00,NaN,NaN
1,16506,1968,0.000000,0.000000,0.000000,1.000000e+00,NaN,NaN
2,21015,1968,0.000000,1.000000,0.000000,5.238103e-17,NaN,NaN
3,21515,1968,0.000000,1.000000,0.000000,0.000000e+00,NaN,NaN
4,2366,1968,0.000000,0.000000,0.000000,9.999853e-01,NaN,NaN
...,...,...,...,...,...,...,...,...
119318,9831,2025,0.000000,0.056709,0.022222,5.905482e-03,NaN,NaN
119319,9834,2025,0.000035,0.073471,0.022222,1.935864e-02,NaN,NaN
119320,9931,2025,0.001887,0.045431,0.044444,1.955908e-03,NaN,NaN
119321,9940,2025,0.080838,0.095844,0.244444,1.407078e-01,NaN,NaN


In [15]:
df_merged['companyid'].isna().count()

np.int64(119323)